### First, load the evaluation wind processes

In [ ]:
import numpy as np
from config import *
from env_utils import get_4wt_symmetric_env, get_lhs_env
from serial_refine_agent import SR_ProportionalController

# Settings
VERBOSE_EVAL = False
RANDOM_AGENT_REPS = 10

from utils import EVAL_WIND_DIRECTIONS, EVAL_WIND_SPEEDS, EVAL_FIXED_WIND_DIRECTIONS, EVAL_FIXED_WIND_SPEEDS, EVAL_RENDER_WIND_DIRECTIONS, EVAL_RENDER_WIND_SPEEDS, EVAL_RENDER_FIXED_WIND_DIRECTIONS, EVAL_RENDER_FIXED_WIND_SPEEDS, evaluate_model, render_model, get_propSR_action, get_noisy_propSR_action, get_model_action, get_nondeterministic_model_action, get_naive_action, get_random_action

def get_base_env(env_type, is_propsr=False, dynamic=False, privileged=False):
    action_representation = "yaw" if is_propsr else "wind"
    dynamic_mode = None if not dynamic else "observation_points"
    if env_type == "4Symm":
        env = get_4wt_symmetric_env(episode_length=EPISODE_LEN, privileged=privileged, changing_wind=True, 
            action_representation=action_representation, load_pyglet_visualization=True, dynamic_mode=dynamic_mode)
    elif env_type == "8LHS":
        env = get_lhs_env("lhs_env_nt8_md150_wb750x750", privileged=privileged, changing_wind=True, 
            load_pyglet_visualization=True, action_representation=action_representation, dynamic_mode=dynamic_mode)
    elif env_type == "16LHS":
        env = get_lhs_env("lhs_env_nt16_md75_wb1500x1500", privileged=privileged, changing_wind=True, 
                load_pyglet_visualization=True, action_representation=action_representation, dynamic_mode=dynamic_mode)
    return env

def get_propsr_agent(env_type):
    if env_type == "4Symm":
        return SR_ProportionalController()
    elif env_type == "8LHS":
        return SR_ProportionalController(
            wind_directions=np.load("data/serial_refine/lhs8_wind_direction.npy"),
            optimal_yaws=np.load("data/serial_refine/lhs8_yaw_angles_opt.npy"),
        )
    elif env_type == "16LHS":
        return SR_ProportionalController(
            wind_directions=np.load("data/serial_refine/lhs16_wind_direction.npy"),
            optimal_yaws=np.load("data/serial_refine/lhs16_yaw_angles_opt.npy"),
        )

def save_scores_and_render(exp_name, env, agent_fn, n_seeds=1,
        wind_directions=EVAL_WIND_DIRECTIONS, wind_speeds=EVAL_WIND_SPEEDS, 
        render_wind_directions=EVAL_RENDER_WIND_DIRECTIONS, render_wind_speeds=EVAL_RENDER_WIND_SPEEDS, 
        EVAL_REPS=EVAL_REPS, EPISODE_LEN=EPISODE_LEN):
    print(f"Evaluating model {exp_name}")
    video = render_model(env, agent_fn, render_wind_directions, render_wind_speeds, EPISODE_LEN)
    video[0].save(f"data/eval/renders/{exp_name}.gif", save_all=True, append_images=video[1:], loop=0, duration=1000/30)
    total_rewards, total_powers = evaluate_model(env, agent_fn, wind_directions, wind_speeds, EVAL_REPS, EPISODE_LEN, N_SEEDS=n_seeds)
    np.save(f"data/eval/scores/{exp_name}_total_rewards.npy", total_rewards)
    np.save(f"data/eval/scores/{exp_name}_total_powers.npy", total_powers)

def save_fw_wind_rose(exp_name, env, agent_fn, n_angles=120, n_seeds=1):
    wind_directions = [[int(angle)] for angle in np.linspace(0, 360, n_angles)]
    wind_speeds = [[8.0] for _ in range(n_angles)]
    print(f"Evaluating model {exp_name}")
    total_rewards, total_powers = evaluate_model(env, agent_fn, wind_directions, wind_speeds, n_angles, EPISODE_LEN, N_SEEDS=n_seeds)
    np.save(f"data/eval/scores/fw_wind_rose/{exp_name}_total_rewards.npy", total_rewards)
    np.save(f"data/eval/scores/fw_wind_rose/{exp_name}_total_powers.npy", total_powers)
    print(f"Shape of total_rewards: {total_rewards.shape}")

### [FW] 4wt fixed wind results | PropSR, Naive, Random

In [ ]:
for env_type in ["4Symm", "8LHS", "16LHS"]:
    # #### Proportional SR 
    save_scores_and_render(f"{env_type}_FW_PropSR", get_base_env(env_type, is_propsr=True), get_propSR_action(get_propsr_agent(env_type)), 
        wind_directions=EVAL_FIXED_WIND_DIRECTIONS, wind_speeds=EVAL_FIXED_WIND_SPEEDS, 
        render_wind_directions=EVAL_RENDER_FIXED_WIND_DIRECTIONS, render_wind_speeds=EVAL_RENDER_FIXED_WIND_SPEEDS)
    #### Naive 
    save_scores_and_render(f"{env_type}_FW_Naive", get_base_env(env_type), get_naive_action(), 
        wind_directions=EVAL_FIXED_WIND_DIRECTIONS, wind_speeds=EVAL_FIXED_WIND_SPEEDS, 
        render_wind_directions=EVAL_RENDER_FIXED_WIND_DIRECTIONS, render_wind_speeds=EVAL_RENDER_FIXED_WIND_SPEEDS)
    #### Random
    save_scores_and_render(f"{env_type}_FW_Random", get_base_env(env_type), get_random_action(), n_seeds=RANDOM_AGENT_REPS,
        wind_directions=EVAL_FIXED_WIND_DIRECTIONS, wind_speeds=EVAL_FIXED_WIND_SPEEDS, 
        render_wind_directions=EVAL_RENDER_FIXED_WIND_DIRECTIONS, render_wind_speeds=EVAL_RENDER_FIXED_WIND_SPEEDS)

### [FW] Wind roses computation

In [ ]:
from serial_refine_agent import SR_ProportionalController
from env_utils import get_4wt_symmetric_env
from config import EPISODE_LEN

for env_type in ["4Symm", "8LHS", "16LHS"]:
    #### Proportional SR 
    save_fw_wind_rose(f"{env_type}_FW_PropSR", get_base_env(env_type, is_propsr=True), get_propSR_action(get_propsr_agent(env_type)))
    #### Naive
    save_fw_wind_rose(f"{env_type}_FW_Naive", get_base_env(env_type), get_naive_action())
    #### Random
    save_fw_wind_rose(f"{env_type}_FW_Random", get_base_env(env_type), get_random_action(), n_seeds=RANDOM_AGENT_REPS)

### [FW] PPO scores and rose 
(local scores are not necessary, but it doesnt take long so i do it anyways)

In [ ]:
from serial_refine_agent import SR_ProportionalController
from env_utils import get_4wt_symmetric_env
from config import EPISODE_LEN
from stable_baselines3 import PPO

models = [
    ("4Symm", PPO.load("data/models/from_server/PPO_4wt_unprivileged_1.00M")),
    ("8LHS", PPO.load("data/models/from_server/PPO_8lhs_unprivileged_1.00M")),
    ("16LHS", PPO.load("data/models/from_server/PPO_16lhs_unprivileged_1.00M")),
]

#### PPO
for env_type, model in models:
    save_scores_and_render(f"{env_type}_FW_PPO_local", get_base_env(env_type), get_model_action(model),
        wind_directions=EVAL_FIXED_WIND_DIRECTIONS, wind_speeds=EVAL_FIXED_WIND_SPEEDS, 
        render_wind_directions=EVAL_RENDER_FIXED_WIND_DIRECTIONS, render_wind_speeds=EVAL_RENDER_FIXED_WIND_SPEEDS)
    save_fw_wind_rose(f"{env_type}_FW_PPO", get_base_env(env_type), get_model_action(model))

In [ ]:
print(np.array(EVAL_FIXED_WIND_DIRECTIONS).shape)

print("NEXT three are new results to read for the first time")
scores = np.load("data/eval/scores/4Symm_FW_PPO_local_total_powers.npy")
print(f"Scores shape: {scores.shape}")
print(f"Mean: {np.mean(scores[0].sum(axis=1))}")
scores = np.load("data/eval/scores/8LHS_FW_PPO_local_total_powers.npy")
print(f"Scores shape: {scores.shape}")
print(f"Mean: {np.mean(scores[0].sum(axis=1))}")
scores = np.load("data/eval/scores/16LHS_FW_PPO_local_total_powers.npy")
print(f"Scores shape: {scores.shape}")
print(f"Mean: {np.mean(scores[0].sum(axis=1))}")



scores = np.load("data/eval/scores/4Symm_FW_PropSR_total_powers.npy")
print(f"Scores shape: {scores.shape}")
print(f"Mean: {np.mean(scores[0].sum(axis=1))}")
scores = np.load("data/eval/scores/4Symm_CW_PropSR_total_powers.npy")
print(f"Scores shape: {scores.shape}")
print(f"Mean: {np.mean(scores[0].sum(axis=1))}")



### [CW] PropSR, NoisyPropSR, Naive, Random

In [ ]:
for env_type in ["4Symm", "8LHS", "16LHS"]:
    #### Proportional SR 
    save_scores_and_render(f"{env_type}_CW_PropSR", get_base_env(env_type, is_propsr=True), get_propSR_action(get_propsr_agent(env_type)))
    #### Proportional SR 
    save_scores_and_render(f"{env_type}_CW_NoisyPropSR", get_base_env(env_type, is_propsr=True), get_noisy_propSR_action(get_propsr_agent(env_type)))
    #### Naive 
    save_scores_and_render(f"{env_type}_CW_Naive", get_base_env(env_type), get_naive_action())
    #### Random
    save_scores_and_render(f"{env_type}_CW_Random", get_base_env(env_type), get_random_action(), n_seeds=RANDOM_AGENT_REPS)

### [CW] PPO models

In [ ]:
# Actually just taken from wandb!

# from stable_baselines3 import PPO
# from env_utils import get_4wt_symmetric_env
# from config import *
# from wind_processes import SetSequenceWindProcess
# from PIL import Image
# import numpy as np
# import tqdm

# model = PPO.load("data/models/from_server/2_PPO_4wt_symmetric_unprivileged_cw_1.00M")
# # model = PPO.load("data/models/from_server/global_action_opt.zip") # action_representation="yaw",
# # model = PPO.load("data/models/from_server/global_action_end.zip") # action_representation="yaw",

# env = get_4wt_symmetric_env(episode_length=EPISODE_LEN, privileged=False, changing_wind=True, 
#     load_pyglet_visualization=True)

# #### Deterministic PPO
# save_scores_and_render("CW_PPO", env, get_model_action(model))
# ### Nondeterministic PPO
# save_scores_and_render("CW_PPO_ND", env, get_nondeterministic_model_action(model))

### [DynOP] 4wt observation points | PropSR, Naive, Random

In [ ]:
for env_type in ["8LHS", "16LHS"]: # (we dont consider 16LHS because it is too slow)
# for env_type in ["8LHS"]:
# for env_type in ["4Symm"]:
    #### Proportional SR 
    save_scores_and_render(f"{env_type}_DynOP_PropSR", get_base_env(env_type, dynamic=True, is_propsr=True), get_propSR_action(get_propsr_agent(env_type)))
    #### Naive
    save_scores_and_render(f"{env_type}_DynOP_Naive", get_base_env(env_type, dynamic=True), get_naive_action()) 
    #### Random
    save_scores_and_render(f"{env_type}_DynOP_Random", get_base_env(env_type, dynamic=True), get_random_action(),  n_seeds=RANDOM_AGENT_REPS)
# env_type = "8LHS"
# env = get_base_env(env_type, dynamic=True)
# env.time_delta = 5
# env.op_per_turbine = 5
# save_scores_and_render(f"{env_type}_DynOP_Naive", env, get_naive_action()) 

### [DynOP] PPO evaluations

In [ ]:
# May not need to be done as we have the data already in wandb!
# Maybe just download it here!